# Experiment4
`PatchCore`

# 1. Load packages

In [1]:
%load_ext autoreload
%autoreload 2

from analysis_tools.common import *
from util import *

import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import tensorflow_probability as tfp
import sklearn
import cv2
import os

np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
sklearn.random.seed(RANDOM_STATE)

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)
    
# strategy = tf.distribute.MirroredStrategy()

ModuleNotFoundError: No module named 'tensorflow_probability'

# 2. Load dataset

## 2.1 screw

In [3]:
class_sample = 'screw'

In [4]:
from sklearn.model_selection import train_test_split

@delayed
def load_img(path, size):
    return cv2.resize(cv2.imread(path), (size, size))

SIZE        = 128
INPUT_SHAPE = (SIZE, SIZE, 3)
train_full_data_meta = pd.read_csv(join(PATH.input, 'train_df.csv'), index_col=0)
train_full_data_meta = train_full_data_meta.query(f"`class` == '{class_sample}'")
paths                = train_full_data_meta['file_name']

In [6]:
train_full_data_meta

,file_name,class,state,label
index,,,,
7,10007.png,screw,good,screw-good
19,10019.png,screw,thread_side,screw-thread_side
35,10035.png,screw,manipulated_front,screw-manipulated_front
44,10044.png,screw,good,screw-good
46,10046.png,screw,good,screw-good
...,...,...,...,...
4223,14223.png,screw,good,screw-good
4234,14234.png,screw,good,screw-good
4246,14246.png,screw,good,screw-good


In [32]:
from sklearn.model_selection import train_test_split

def copy(class_sample, state, data_id, paths, target_dir):
    dst_dir = join(target_dir, class_sample, data_id, state)
    mkdir(dst_dir)
    for path in paths:
        shutil.copy(join(PATH.train, path), join(dst_dir, get_name(path)))


target_dir = join(PATH.root, 'anomalib', 'datasets', 'MVTec')
for state in tqdm(train_full_data_meta['state'].unique()):
    paths = [join(PATH.train, name) for name in train_full_data_meta.query(f"state == '{state}'")['file_name']]
    
    if state == 'good':
#         paths = paths[:int(len(paths)/3)]
        train_paths, test_paths = train_test_split(paths)
        for data_id, paths in zip(['train', 'test'], [train_paths, test_paths]):
            copy(class_sample, state, data_id, paths, target_dir)
    else:
        train_paths, test_paths = [], paths
        data_id = 'test'
        copy(class_sample, state, data_id, paths, target_dir)

100%|██████████| 6/6 [00:02<00:00,  2.23it/s]


In [31]:
rmdir(target_dir)